In [2]:
import re

import sqlalchemy as sql

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../../data/relatorio.csv', sep=';')
df.head()

,CNPJ do gerador,Razão Social do gerador,Estado,Município,Código da Categoria,Categoria de Atividade,Código do Detalhe,Detalhe,Ano da geração,Cód. Resíduo,Tipo de Resíduo,Quantidade Gerada,Unidade,Classificação Resíduo,Situação Cadastral
0,00.001.388/0002-26,DISTRIBUIDORA BRASILIA DE VEICULOS S/A,DISTRITO FEDERAL,BRASILIA,18,"Transporte, Terminais, Depósitos e Comércio",6,Comércio de combustíveis e derivados de petróleo,2016,160799,Outros resíduos não anteriormente especificados,"2.034,43",kilogramas,Não Perigoso,Encerrado
1,00.001.388/0002-26,DISTRIBUIDORA BRASILIA DE VEICULOS S/A,DISTRITO FEDERAL,BRASILIA,18,"Transporte, Terminais, Depósitos e Comércio",6,Comércio de combustíveis e derivados de petróleo,2016,200121,"Lâmpadas fluorescentes, de vapor de sódio e me...","80,00",Unidade,Perigoso,Encerrado
2,00.001.388/0002-26,DISTRIBUIDORA BRASILIA DE VEICULOS S/A,DISTRITO FEDERAL,BRASILIA,18,"Transporte, Terminais, Depósitos e Comércio",6,Comércio de combustíveis e derivados de petróleo,2016,130899,Outros resíduos não anteriormente especificados,"538,54",kilogramas,Perigoso,Encerrado
3,00.001.388/0002-26,DISTRIBUIDORA BRASILIA DE VEICULOS S/A,DISTRITO FEDERAL,BRASILIA,18,"Transporte, Terminais, Depósitos e Comércio",6,Comércio de combustíveis e derivados de petróleo,2016,130506,Óleos provenientes dos separadores óleo/água,"4.300,00",Litro,Perigoso,Encerrado
4,00.001.388/0002-26,DISTRIBUIDORA BRASILIA DE VEICULOS S/A,DISTRITO FEDERAL,BRASILIA,18,"Transporte, Terminais, Depósitos e Comércio",6,Comércio de combustíveis e derivados de petróleo,2016,130507,Água com óleo proveniente dos separadores óleo...,"10.800,00",Litro,Perigoso,Encerrado


In [4]:
df['cnpj'] = df['CNPJ do gerador'].apply(lambda cnpj: ''.join(digit for digit in cnpj if digit.isdigit()))

In [5]:
df_relevante = df[['cnpj', 'Quantidade Gerada', 'Unidade', 'Ano da geração']]

In [6]:
def parse_qtde(qtde):
    if not re.match(r'(\d+\.?)*\d+,\d+', qtde):
        return None
    
    return float(qtde.replace('.', '').replace(',', '.'))

df_relevante['qtde_residuo'] = df_relevante['Quantidade Gerada'].apply(parse_qtde)

/tmp/ipykernel_246645/944466757.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevante['qtde_residuo'] = df_relevante['Quantidade Gerada'].apply(parse_qtde)


In [7]:
df_agrupado_por_ano = df_relevante.groupby(['cnpj', 'Unidade', 'Ano da geração']).sum().reset_index()

/tmp/ipykernel_246645/919026996.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agrupado_por_ano = df_relevante.groupby(['cnpj', 'Unidade', 'Ano da geração']).sum().reset_index()


In [8]:
df_massa_e_volume = df_agrupado_por_ano[df_agrupado_por_ano['Unidade'] != 'Unidade'].copy()

In [10]:
df_massa_e_volume['qtde_padronizada'] = df_massa_e_volume['qtde_residuo'] / 1000
df_massa_e_volume['unidade_padronizada'] = df_massa_e_volume['Unidade'].map({'kilogramas': 'ton/ano', 'Litro': 'm3/ano'})
df_massa_e_volume['tipo_unidade_padronizada'] = df_massa_e_volume['unidade_padronizada'].map({'ton/ano': 'massa', 'm3/ano': 'volume'})

In [11]:
df_massa_e_volume[['cnpj', 'Ano da geração', 'tipo_unidade_padronizada', 'qtde_padronizada', 'unidade_padronizada']].head()

,cnpj,Ano da geração,tipo_unidade_padronizada,qtde_padronizada,unidade_padronizada
0,00001388000226,2016,volume,15.10000,m3/ano
1,00001388000226,2017,volume,7.20000,m3/ano
2,00001388000226,2018,volume,14.40000,m3/ano
3,00001388000226,2019,volume,9.30000,m3/ano
8,00001388000226,2016,massa,2.57297,ton/ano


In [12]:
df_ibama_final = df_massa_e_volume[['cnpj', 'Ano da geração', 'tipo_unidade_padronizada', 'qtde_padronizada', 'unidade_padronizada']]

In [13]:
def categorizar_faixa_massa_residuo(y: float):
    limites = [
        (0, 1000),
        (1000, 5000),
        (5000,10000),
        (10000,20000),
        (20000,40000),
        (40000,80000),
        (80000,160000),
        (160000,320000),
        (320000,640000),
        (640000,float('inf')),
    ]
    
    for i, limite in enumerate(limites):
        lim_inferior, lim_superior = limite
        if lim_inferior < y < lim_superior:
            return i
    return -1

In [14]:
df_ibama_final['faixa_massa_residuo'] = df_ibama_final['qtde_padronizada'].apply(categorizar_faixa_massa_residuo)
df_ibama_final = df_ibama_final.rename(columns={'Ano da geração': 'ano'})
df_ibama_final.head()

/tmp/ipykernel_246645/3935948677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ibama_final['faixa_massa_residuo'] = df_ibama_final['qtde_padronizada'].apply(categorizar_faixa_massa_residuo)


,cnpj,ano,tipo_unidade_padronizada,qtde_padronizada,unidade_padronizada,faixa_massa_residuo
0,00001388000226,2016,volume,15.10000,m3/ano,0
1,00001388000226,2017,volume,7.20000,m3/ano,0
2,00001388000226,2018,volume,14.40000,m3/ano,0
3,00001388000226,2019,volume,9.30000,m3/ano,0
8,00001388000226,2016,massa,2.57297,ton/ano,0


In [15]:
df_ibama_final.to_csv('../../data/final_ibama.csv', index=False)